In [2]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlflow.models.signature import infer_signature
import pandas as pd

# Configura el URI del servidor de seguimiento de MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("TiempoEntrega")

# Datos de ejemplo simulados
data = {
    "distancia_km": [5, 10, 15, 20, 25],
    "numero_articulos": [1, 3, 5, 2, 4],
    "velocidad_kmh": [30, 40, 35, 25, 45],
    "tiempo_entrega_min": [10, 20, 35, 50, 55]  # Objetivo
}
df = pd.DataFrame(data)

# Convertir las columnas a float64 si hay valores faltantes
X = df[["distancia_km", "numero_articulos", "velocidad_kmh"]].astype("float64")
y = df["tiempo_entrega_min"]

# Escalar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Dividir los datos en entrenamiento y prueba, con un ajuste en el tamaño del conjunto de prueba
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=42)

# Asegurarnos de que el conjunto de prueba tiene al menos 2 muestras
if len(X_test) < 2:
    # Ajustamos el tamaño del conjunto de prueba (si es necesario)
    test_size = 0.5  # Cambiar a 50% si los datos son pocos
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=42)

    # Verificamos nuevamente
    if len(X_test) < 2:
        raise ValueError("El conjunto de prueba debe contener al menos dos muestras para calcular métricas como R^2.")

# Entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Calcular métricas de validación
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

# Crear input_example y la firma
input_example = X.iloc[:5]  # Usar los datos originales no escalados para input_example
signature = infer_signature(X, model.predict(X_scaled))

# Registrar el modelo en MLflow
with mlflow.start_run() as run:
    mlflow.log_param("fit_intercept", model.fit_intercept)
    mlflow.log_metric("train_score", train_score)
    mlflow.log_metric("test_score", test_score)

    # Registrar el modelo con input_example y signature
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="Tiempo",
        input_example=input_example,
        signature=signature
    )
    # Apuntador a las librerias
    mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path="model",
    registered_model_name="Tiempo",
    input_example=input_example,
    signature=signature,
    conda_env="mlops_env.yaml"  # Ruta al archivo conda.yaml
    )


print(f"Modelo registrado correctamente con ID: {run.info.run_id}")


Registered model 'Tiempo' already exists. Creating a new version of this model...
2025/01/29 00:04:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Tiempo, version 19
Created version '19' of model 'Tiempo'.
Registered model 'Tiempo' already exists. Creating a new version of this model...
2025/01/29 00:04:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Tiempo, version 20


🏃 View run masked-shark-546 at: http://127.0.0.1:5000/#/experiments/11/runs/2a668ccca23c483988a37764e56fb975
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/11
Modelo registrado correctamente con ID: 2a668ccca23c483988a37764e56fb975


Created version '20' of model 'Tiempo'.
